In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.transforms import transforms

import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Normalization values
def calculate_mean_std(traindata):
    images = torch.stack([img for img, _ in traindata], dim=3)
    mean = images.view(3, -1).mean(dim=1)
    std = images.view(3, -1).std(dim=1)
    return mean, std

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
traindata = torchvision.datasets.ImageFolder('data/train/', transform=transform)

mean, std = calculate_mean_std(traindata)

In [ ]:
# train data normalized
transform = transforms.Compose([
    transforms.Resize(800),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

traindata = torchvision.datasets.ImageFolder('data/train/', transform=transform)

# Split train data
trainsize = int(718*.8)
validationsize = 718 - trainsize
trainset, validationset = torch.utils.data.random_split(traindata, lengths=[trainsize, validationsize])

In [ ]:
# train and validation loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=32)

In [ ]:
# test set and test loader
testset = torchvision.datasets.ImageFolder('test_small/', transform=transform)
testloader = torch.utils.data.DataLoader(testset)#, batch_size=32)

In [ ]:
# visualization
dataiter = iter(trainloader)
images, labels = dataiter.next()

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
plt.figure(figsize=(20, 20))
plt.axis('off')
imshow(torchvision.utils.make_grid(images))
print(labels)

In [6]:
# model

# flattening class
class Flatten(nn.Module):
    def forward(self, x):
        # print(x.shape)
        return x.view(x.size(0), -1)


model = nn.Sequential(
    nn.Conv2d(3, 8, 5),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(8, 16, 5),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(16, 32, 5),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2, 2),
    Flatten(),
    nn.Linear(64 * 40 * 40, 1000),
    nn.ReLU(),
    nn.Linear(1000, 2)
)

model

Sequential(
  (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Flatten()
  (9): Linear(in_features=102400, out_features=1000, bias=True)
  (10): ReLU()
  (11): Linear(in_features=1000, out_features=2, bias=True)
)

In [9]:
# Randomly initialized model
x = images
out = model(x)
_, pred= torch.max(out, dim=1)
print(f'Pred: {pred}')
print(f'Ground: {labels}')

Pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
Ground: tensor([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 0, 1, 1])


In [7]:
# objective and optimization functions
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [8]:
for epoch in range(50):  # loop models/r the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() # update

        # print statistics
        running_loss += loss.item()
        
        
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.698
[1,     2] loss: 0.692
[1,     3] loss: 0.694
[1,     4] loss: 0.691
[1,     5] loss: 0.694
[1,     6] loss: 0.689
[1,     7] loss: 0.693
[1,     8] loss: 0.690
[1,     9] loss: 0.685
[1,    10] loss: 0.685
[1,    11] loss: 0.690
[1,    12] loss: 0.686
[1,    13] loss: 0.684
[1,    14] loss: 0.685
[1,    15] loss: 0.679
[1,    16] loss: 0.684
[1,    17] loss: 0.686
[1,    18] loss: 0.683
[2,     1] loss: 0.683
[2,     2] loss: 0.673
[2,     3] loss: 0.678
[2,     4] loss: 0.676
[2,     5] loss: 0.684
[2,     6] loss: 0.666
[2,     7] loss: 0.677
[2,     8] loss: 0.682
[2,     9] loss: 0.667
[2,    10] loss: 0.668
[2,    11] loss: 0.683
[2,    12] loss: 0.675
[2,    13] loss: 0.665
[2,    14] loss: 0.673
[2,    15] loss: 0.661
[2,    16] loss: 0.673
[2,    17] loss: 0.676
[2,    18] loss: 0.671
[3,     1] loss: 0.666
[3,     2] loss: 0.653
[3,     3] loss: 0.661
[3,     4] loss: 0.661
[3,     5] loss: 0.676
[3,     6] loss: 0.641
[3,     7] loss: 0.660
[3,     8] 

In [11]:
# test data probabilities of having tuberculosis i.e 1
probabilities = []
for i in range(len(testset)):
    out = model(testset[i][0].unsqueeze(0))
    probabilities.append(round(out[0][1].item(), 2))
    #prob, _= torch.max(out, dim=1)
    #print(prob)
    #probabilities.append(round(prob.item(), 2))

In [12]:
# id probability map based on image order in testset
id_probability_map = {testset.imgs[i][0][:-4].split("\\")[-1]: probabilities[i] for i in range(len(testset))}

In [13]:
# Exporting
import pandas as pd


def exporting(id_probability_map, submission_number):
    submission = pd.read_csv("Test.csv")
    probs = [id_probability_map[submission['ID'][i]] for i in range(len(submission))]
    final = pd.DataFrame({'ID': list(submission['ID']), 'LABEL': probs})
    final.to_csv('Submission_'+str(submission_number)+'.csv', index=False)

In [14]:
exporting(id_probability_map, 5)

In [15]:
torch.save(model.state_dict(), 'models/simple_model_more_epochs.pt')

# Bigger Model and training on GPU 

In [7]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [8]:
# flattening function
def flatten(x):
        # print(x.shape)
        return x.view(x.size(0), -1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # convolution layers
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 8, 5),
            nn.MaxPool2d(2, 2),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 5),
            nn.MaxPool2d(2, 2),
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 5),
            nn.MaxPool2d(2, 2),
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 64, 5),
            nn.MaxPool2d(2, 2),
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 128, 5),
            nn.MaxPool2d(2, 2),
        )
        
        # dropout
        self.dropout = nn.Dropout(p=.2)
        
        # linear layers
        self.fc1 = nn.Linear(128 * 18 * 18, 1000)
        self.fc2 = nn.Linear(1000, 2)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = flatten(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(self.fc2(x))
        
        return x
        
model = Net()
model.to(device)

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=41472, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_f

In [9]:
# objective and optimization functions
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
for epoch in range(50):  # loop models/r the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() # update

        # print statistics
        running_loss += loss.item()
        
        
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.695
[1,     2] loss: 0.691
[1,     3] loss: 0.689
[1,     4] loss: 0.700
[1,     5] loss: 0.695
[1,     6] loss: 0.691
[1,     7] loss: 0.691
[1,     8] loss: 0.688
[1,     9] loss: 0.694
[1,    10] loss: 0.690
[1,    11] loss: 0.697
[1,    12] loss: 0.694
[1,    13] loss: 0.693
[1,    14] loss: 0.697
[1,    15] loss: 0.694
[1,    16] loss: 0.694
[1,    17] loss: 0.692
[1,    18] loss: 0.689
[2,     1] loss: 0.689
[2,     2] loss: 0.688
[2,     3] loss: 0.686
[2,     4] loss: 0.698
[2,     5] loss: 0.690
[2,     6] loss: 0.689
[2,     7] loss: 0.686
[2,     8] loss: 0.690
[2,     9] loss: 0.693
[2,    10] loss: 0.692
[2,    11] loss: 0.693
[2,    12] loss: 0.702
[2,    13] loss: 0.691
[2,    14] loss: 0.697
[2,    15] loss: 0.698
[2,    16] loss: 0.694
[2,    17] loss: 0.692
[2,    18] loss: 0.692
[3,     1] loss: 0.686
[3,     2] loss: 0.688
[3,     3] loss: 0.686
[3,     4] loss: 0.702
[3,     5] loss: 0.690
[3,     6] loss: 0.695
[3,     7] loss: 0.688
[3,     8] 

In [11]:
torch.save(model.state_dict(), 'models/complex_model_gpu.pt')

In [12]:
model = Net()
model.load_state_dict(torch.load('models/complex_model_gpu.pt', map_location='cpu'), strict=False)
model.eval()

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=41472, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_f

In [13]:
# test data probabilities of having tuberculosis i.e 1
probabilities = []
for i in range(len(testset)):
    out = model(testset[i][0].unsqueeze(0))
    probabilities.append(round(out[0][1].item(), 2))

In [14]:
# id probability map based on image order in testset
id_probability_map = {testset.imgs[i][0][:-4].split("\\")[-1]: probabilities[i] for i in range(len(testset))}

In [15]:
# Exporting
import pandas as pd


def exporting(id_probability_map, submission_number):
    submission = pd.read_csv("Test.csv")
    probs = [id_probability_map[submission['ID'][i]] for i in range(len(submission))]
    final = pd.DataFrame({'ID': list(submission['ID']), 'LABEL': probs})
    final.to_csv('Submission_'+str(submission_number)+'.csv', index=False)

In [16]:
exporting(id_probability_map, 4)